# <font color='blue'>Data Science Academy</font>
# <font color='blue'>Deep Learning Frameworks</font>

## Mini-Projeto 3

In [1]:
# Versão da Linguagem Python
from platform import python_version
print('Versão da Linguagem Python Usada Neste Jupyter Notebook:', python_version())

Versão da Linguagem Python Usada Neste Jupyter Notebook: 3.7.6


## Inteligência Artificial Para Detectar Depressão em Comentários nas Redes Sociais

![Mini-Projeto3](imagens/imagem.png)

As redes sociais foram desenvolvidas como um excelente ponto de encontro para seus usuários se comunicarem com seus amigos e compartilharem suas opiniões, fotos e vídeos refletindo seus humores, sentimentos e atividades.

Mas infelizmente nos últimos anos as redes sociais tornaram-se um ambiente tóxico, onde os piores sentimentos costumam ser demonstrados das piores formas possíveis e com as fake news em boa parte da comunicação que circula nas redes, o que tem levado a uma onda de depressão entre s usuários, algo sem precedentes na história recente das redes sociais.

Isso cria uma oportunidade para analisar dados de redes sociais em busca de sentimentos do usuário para investigar seus humores e atitudes quando eles estão se comunicando por meio dessas ferramentas on-line.

Neste Mini-Projeto, objetivamos realizar análises de depressão em dados do Twitter. Para investigar o efeito da detecção de depressão, usaremos a técnica de Deep Learning para Processamento de Linguagem Natural como um método eficiente e escalável e com uma arquitetura híbrida de modelo CNN e LSTM.

Parte do conhecimento neste Mini-Projeto será estudado em detalhes no curso de Processamento de Linguagem Natural da Formação IA, mas você encontra nos comentários em cada célula do Jupyter Notebook em anexo tudo que é necessário para reproduzir o projeto.

Boa aula.

Execute cada célula, leia os comentários e faça modificações se desejar.

In [2]:
!nvidia-smi

Fri May 29 21:04:06 2020       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 440.64.00    Driver Version: 440.64.00    CUDA Version: 10.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|===============================+======================+======================|
|   0  TITAN X (Pascal)    On   | 00000000:05:00.0 Off |                  N/A |
| 24%   42C    P8     9W / 250W |    115MiB / 12194MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   1  GeForce GTX 108...  On   | 00000000:09:00.0 Off |                  N/A |
| 23%   40C    P8     9W / 250W |      2MiB / 11178MiB |      0%      Default |
+-------------------------------+----------------------+----------------------+
|   2  T

In [3]:
# Para atualizar um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install -U nome_pacote

# Para instalar a versão exata de um pacote, execute o comando abaixo no terminal ou prompt de comando:
# pip install nome_pacote==versão_desejada

# Depois de instalar ou atualizar o pacote, reinicie o jupyter notebook.

# Instala o pacote watermark. 
# Esse pacote é usado para gravar as versões de outros pacotes usados neste jupyter notebook.
!pip install -q -U watermark

In [4]:
# Instala o tweepy
!pip install -q tweepy

In [5]:
# Pacote para criação de texto fixo
!pip install -q ftfy

In [6]:
# Pacote para tratamento de texto
!pip install -q textblob

In [7]:
# Pacote de PLN
!pip install -q gensim

In [8]:
# Imports
import csv
import re
import ftfy
import nltk
import tweepy
import textblob
import sklearn
import gensim
import keras
import tensorflow
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from nltk import PorterStemmer
from textblob import TextBlob
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from gensim.models import KeyedVectors
from keras.callbacks import EarlyStopping
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential
from keras.layers import Conv1D, Dense, LSTM, Embedding, Dropout, MaxPooling1D

Using TensorFlow backend.


In [9]:
# Versões dos pacotes usados neste jupyter notebook
%reload_ext watermark
%watermark -a "Data Science Academy" --iversions

ftfy       5.7
textblob   0.15.3
nltk       3.4.5
csv        1.0
tensorflow 2.2.0
tweepy     3.8.0
sklearn    0.23.1
numpy      1.18.4
pandas     1.0.3
gensim     3.8.3
re         2.2.1
keras      2.3.1
Data Science Academy


## Checando o Hardware Disponível no Servidor da DSA - CPU e GPUs

In [10]:
# Lista todos os dispositivos disponiveis
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 6199313257260956127
, name: "/device:XLA_CPU:0"
device_type: "XLA_CPU"
memory_limit: 17179869184
locality {
}
incarnation: 10740029211206724038
physical_device_desc: "device: XLA_CPU device"
, name: "/device:XLA_GPU:0"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9508734379612702331
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:1"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 14461824057884254759
physical_device_desc: "device: XLA_GPU device"
, name: "/device:XLA_GPU:2"
device_type: "XLA_GPU"
memory_limit: 17179869184
locality {
}
incarnation: 9847639884190807341
physical_device_desc: "device: XLA_GPU device"
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 23702262272
locality {
  bus_id: 1
  links {
  }
}
incarnation: 8598146507053718304
physical_device_desc: "device: 0, name: TITAN RTX, pci bus id: 00

> Se por acaso não aparecer para você todas as GPUs, reinstale o TensorFlow (execute pelo console e não pelo Jupyter):

In [11]:
# pip uninstall tensorflow
# pip install tensorflow

In [12]:
import tensorflow as tf
print("Número Disponível de GPUs: ", len(tf.config.experimental.list_physical_devices('GPU')))

Número Disponível de GPUs:  3


In [13]:
# Lista o código de cada GPU
tf.config.list_physical_devices('GPU')

[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU'),
 PhysicalDevice(name='/physical_device:GPU:2', device_type='GPU')]

## Carregando o Dataset

O arquivo csv fornecido com este Mini-Projeto é genérico e deve ser usado somente para testar este modelo. Você pode depois alterar o projeto e coletar dados do Tweet em tempo real, se desejar. Caso tenha conta no Twitter, você também pode acessar seu perfil e baixar todos os seus tweets desde que a conta foi criada.

In [14]:
# Define o nome do arquivo
arquivo = "dados/dataset.csv"

In [15]:
# Definimos uma variável para extrair registros de forma aleatória do arquivo de dados
num_random_linhas = 20000

In [16]:
# Reproducibilidade
np.random.seed(1234)

In [17]:
# Carrega o dataset como se estivesse coletando dados do Twitter em tempo real
df_tweets = pd.read_csv(arquivo, encoding = "UTF-8", usecols = range(0, 4), nrows = num_random_linhas)

In [18]:
# Visualiza os dados
df_tweets.head()

,randomness,timestamp,tweet,label
0,8586,31-10-2019 19:25,"b'Harley Quinn, baby!!\n.\n.\n.\n#happyhallowe...",1
1,13304,30-10-2019 19:52,b'RT @CINetwork2019: @aronseal @TorontoPolice ...,0
2,13492,01-11-2019 09:30,b'RT @CreateTheRipple: Be crazy. Be silly. Be ...,1
3,17567,28-10-2019 23:33,b'RT @ManuelaMravlak: Here is a small sample o...,1
4,17790,30-10-2019 02:31,b'RT @MiaStJohnBoxer: After I lost my son &amp...,0


In [19]:
# Shape
df_tweets.shape

(20000, 4)

## Limpeza e Preparação do Dataset

In [20]:
# Expandindo as contrações, comuns no texto em inglês
cList = {
  "ain't": "am not",
  "aren't": "are not",
  "can't": "cannot",
  "can't've": "cannot have",
  "'cause": "because",
  "could've": "could have",
  "couldn't": "could not",
  "couldn't've": "could not have",
  "didn't": "did not",
  "doesn't": "does not",
  "don't": "do not",
  "hadn't": "had not",
  "hadn't've": "had not have",
  "hasn't": "has not",
  "haven't": "have not",
  "he'd": "he would",
  "he'd've": "he would have",
  "he'll": "he will",
  "he'll've": "he will have",
  "he's": "he is",
  "how'd": "how did",
  "how'd'y": "how do you",
  "how'll": "how will",
  "how's": "how is",
  "I'd": "I would",
  "I'd've": "I would have",
  "I'll": "I will",
  "I'll've": "I will have",
  "I'm": "I am",
  "I've": "I have",
  "isn't": "is not",
  "it'd": "it had",
  "it'd've": "it would have",
  "it'll": "it will",
  "it'll've": "it will have",
  "it's": "it is",
  "let's": "let us",
  "ma'am": "madam",
  "mayn't": "may not",
  "might've": "might have",
  "mightn't": "might not",
  "mightn't've": "might not have",
  "must've": "must have",
  "mustn't": "must not",
  "mustn't've": "must not have",
  "needn't": "need not",
  "needn't've": "need not have",
  "o'clock": "of the clock",
  "oughtn't": "ought not",
  "oughtn't've": "ought not have",
  "shan't": "shall not",
  "sha'n't": "shall not",
  "shan't've": "shall not have",
  "she'd": "she would",
  "she'd've": "she would have",
  "she'll": "she will",
  "she'll've": "she will have",
  "she's": "she is",
  "should've": "should have",
  "shouldn't": "should not",
  "shouldn't've": "should not have",
  "so've": "so have",
  "so's": "so is",
  "that'd": "that would",
  "that'd've": "that would have",
  "that's": "that is",
  "there'd": "there had",
  "there'd've": "there would have",
  "there's": "there is",
  "they'd": "they would",
  "they'd've": "they would have",
  "they'll": "they will",
  "they'll've": "they will have",
  "they're": "they are",
  "they've": "they have",
  "to've": "to have",
  "wasn't": "was not",
  "we'd": "we had",
  "we'd've": "we would have",
  "we'll": "we will",
  "we'll've": "we will have",
  "we're": "we are",
  "we've": "we have",
  "weren't": "were not",
  "what'll": "what will",
  "what'll've": "what will have",
  "what're": "what are",
  "what's": "what is",
  "what've": "what have",
  "when's": "when is",
  "when've": "when have",
  "where'd": "where did",
  "where's": "where is",
  "where've": "where have",
  "who'll": "who will",
  "who'll've": "who will have",
  "who's": "who is",
  "who've": "who have",
  "why's": "why is",
  "why've": "why have",
  "will've": "will have",
  "won't": "will not",
  "won't've": "will not have",
  "would've": "would have",
  "wouldn't": "would not",
  "wouldn't've": "would not have",
  "y'all": "you all",
  "y'alls": "you alls",
  "y'all'd": "you all would",
  "y'all'd've": "you all would have",
  "y'all're": "you all are",
  "y'all've": "you all have",
  "you'd": "you had",
  "you'd've": "you would have",
  "you'll": "you you will",
  "you'll've": "you you will have",
  "you're": "you are",
  "you've": "you have"
}

In [21]:
# Com expressões regulares, separamos as contrações com o símbolo pipe
c_re = re.compile('(%s)' % '|'.join(cList.keys()))

In [22]:
# Visualiza a lista de contrações de palavras
c_re

re.compile(r"(ain't|aren't|can't|can't've|'cause|could've|couldn't|couldn't've|didn't|doesn't|don't|hadn't|hadn't've|hasn't|haven't|he'd|he'd've|he'll|he'll've|he's|how'd|how'd'y|how'll|how's|I'd|I'd've|I'll|I'll've|I'm|I've|isn't|it'd|it'd've|it'll|it'll've|it's|let's|ma'am|mayn't|might've|mightn't|mightn't've|must've|mustn't|mustn't've|needn't|needn't've|o'clock|oughtn't|oughtn't've|shan't|sha'n't|shan't've|she'd|she'd've|she'll|she'll've|she's|should've|shouldn't|shouldn't've|so've|so's|that'd|that'd've|that's|there'd|there'd've|there's|they'd|they'd've|they'll|they'll've|they're|they've|to've|wasn't|we'd|we'd've|we'll|we'll've|we're|we've|weren't|what'll|what'll've|what're|what's|what've|when's|when've|where'd|where's|where've|who'll|who'll've|who's|who've|why's|why've|will've|won't|won't've|would've|wouldn't|wouldn't've|y'all|y'alls|y'all'd|y'all'd've|y'all're|y'all've|you'd|you'd've|you'll|you'll've|you're|you've)",
re.UNICODE)

In [23]:
# Função para expandir as contrações de texto
def funcExpandContracao(text, c_re = c_re):
    def replace(match):
        return cList[match.group(0)]
    return c_re.sub(replace, text)

In [24]:
# Função para limpeza dos tweets
def funcLimpaTweets(tweets):
    
    # Lista para os tweets limpos
    cleaned_tweets = []
    
    # Loop pelos tweets
    for tweet in tweets:
        
        # Converte o tweet sm string
        tweet = str(tweet)
        
        # Se o tweet não tiver URL (sinal barra duas vezes: //) e for maior que 10 caracteres, iniciamos a limpeza
        if re.match("(\w+:\/\/\S+)", tweet) == None and len(tweet) > 10:
            
            # Removemos hashtag, @mention, emoji e URLs de imagens
            tweet = ' '.join(
                re.sub("(@[A-Za-z0-9]+)|(\#[A-Za-z0-9]+)|(<Emoji:.*>)|(pic\.twitter\.com\/.*)", " ", tweet).split())
            
            # Fixamos texto que parecer estranho
            tweet = ftfy.fix_text(tweet)
            
            # Expandimos as contrações de texto
            tweet = funcExpandContracao(tweet)
            
            # Removemos pontuação
            tweet = ' '.join(re.sub("([^0-9A-Za-z \t])", " ", tweet).split())
            
            # Definimos as stop words
            stop_words = set(stopwords.words('english'))
            
            # Tokenizamos o tweet
            word_tokens = nltk.word_tokenize(tweet)
            
            # Filtramos as sentenças com os tokens
            filtered_sentence = [w for w in word_tokens if not w in stop_words]
            
            # Geramos o tweet limpo
            tweet = ' '.join(filtered_sentence)
            
            # Aplicamos Steming (é estudado no curso de PLN)
            tweet = PorterStemmer().stem(tweet)
            
            # Gravamos o tweet limpo
            cleaned_tweets.append(tweet)
            
    return cleaned_tweets

In [25]:
# Transformamos o dataframe em um array para facilitar a limpeza
df_tweets_arr = [x for x in df_tweets["tweet"]]

In [26]:
# Aplicamos a função de limpeza ao dataset de tweets
df_tweets_clean = funcLimpaTweets(df_tweets_arr)

In [27]:
# Visualizamos os dados
df_tweets_clean

['b harley quinn baby n n n n london united ki https co 5vdev2ztbz',
 'b rt only responsible mental health n nlaying blame political wants xe2 x80 xa6',
 'b rt be crazy be silly be be https co dzneklqmbp',
 'b rt here small sample wonderful much roar water impressive little lake tarvisio',
 'b rt after i lost son father i lost took family i refuse let xe2 x80 xa6',
 'b im superman im n n n https co ieqq3vs8n',
 'b rt canis bong13 13 1503',
 'b rt taking charge possible right learning tools resources nin learning section ment xe2 x80 xa6',
 'b mudmaster dual display sports watch n n https co mzltkyfr0m',
 'b https co lsbjvisixd tonialavedrafishingg caught our https co 2p4lowjl7v',
 'b what awesome trending music game nvery funny exicting challenge ncheck out tiles hop n https co btuseolhcb',
 'b rt your mental health priority your happiness essential your self care necessity n n n xe2 x80 xa6',
 'b rt shes fired https co 30aff09kgz',
 'b rt rt rt rt rt rt dino lov',
 'b rt staring tomor

In [28]:
# Vamos preparar um arquivos para salvar os dados em disco
csvFile = open("dados/dataset_clean.csv", 'w', newline = '')
csvWriter = csv.writer(csvFile)
csvWriter.writerow(["Tweets"])

8

In [29]:
# Para cada item da lista de tweets limpos, adicionamos ao arquivo e salvamos em disco
for i in range(len(df_tweets_clean)):
    csvFile = open("dados/dataset_clean.csv", 'a', newline = '')
    csvWriter = csv.writer(csvFile)
    csvWriter.writerow([df_tweets_clean[i]])

In [30]:
# Verifica se o arquivo foi salvo em disco
!ls -la dados

total 5476
drwxrwxr-x 2 dmpm dmpm    4096 May 29 18:34 .
drwxrwxr-x 6 dmpm dmpm    4096 May 29 21:03 ..
-rw-rw-r-- 1 dmpm dmpm 1556788 May 29 21:04 dataset_clean.csv
-rw-rw-r-- 1 dmpm dmpm 4037540 May 29 17:35 dataset.csv


## Pré-Processamento dos Dados

Agora transformamos o dataframe de texto em uma matriz numérica representando o contexto das palavras.

In [31]:
# Parâmetros de controle
MAX_SEQUENCE_LENGTH = 140
MAX_NB_WORDS = 20000
EMBEDDING_DIM = 300

Usaremos o arquivo de embeddings fornecido pelo Google. O arquivo tem mais de 3.5 GB e pode ser usado diretamente no Titan, servidor da DSA, no caminho definido na célula abaixo.

https://code.google.com/archive/p/word2vec/

In [32]:
# Arquivo de embeddings do Google
EMBEDDING_FILE = '/media/datasets/DeepLearningFrameworks/Mini-Projeto3/GoogleNews-vectors-negative300.bin'

In [33]:
# Cria modelo word2vec
word2vec = KeyedVectors.load_word2vec_format(EMBEDDING_FILE, binary = True)

In [34]:
# Cria o tokenizador
tokenizer = Tokenizer(num_words = MAX_NB_WORDS)

In [35]:
# Treina o tokenizador no dataset de tweets limpos
tokenizer.fit_on_texts(df_tweets_clean)

In [36]:
# Extrai as sequências do dataset usando o tokenizador
sequences_c = tokenizer.texts_to_sequences(df_tweets_clean)

In [37]:
# Extrai o índice
word_index = tokenizer.word_index

In [38]:
print('Encontrados %s tokens únicos' % len(word_index))

Encontrados 26562 tokens únicos


In [39]:
# Ajusta as sequências para que fiquem do mesmo tamanho
data_c = pad_sequences(sequences_c, maxlen = MAX_SEQUENCE_LENGTH)

In [40]:
print('Shape do tensor:', data_c.shape)

Shape do tensor: (19999, 140)


In [41]:
# Número de palavras
nb_words = min(MAX_NB_WORDS, len(word_index))

In [42]:
# Cria a matriz de embeddings preenchida com zeros
embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))

In [43]:
# Aplica a função word2vec e gera a matriz de embeddings
for (word, idx) in word_index.items():
    if word in word2vec.vocab and idx < MAX_NB_WORDS:
        embedding_matrix[idx] = word2vec.word_vec(word)

## Separação dos Dados em Treino, Validação e Teste

In [44]:
# Hiperparâmetros
TRAIN_SPLIT = 0.7
TEST_SPLIT = 0.2
LEARNING_RATE = 0.1
EPOCHS = 15

In [45]:
# Visualizando os dados
df_tweets.head()

,randomness,timestamp,tweet,label
0,8586,31-10-2019 19:25,"b'Harley Quinn, baby!!\n.\n.\n.\n#happyhallowe...",1
1,13304,30-10-2019 19:52,b'RT @CINetwork2019: @aronseal @TorontoPolice ...,0
2,13492,01-11-2019 09:30,b'RT @CreateTheRipple: Be crazy. Be silly. Be ...,1
3,17567,28-10-2019 23:33,b'RT @ManuelaMravlak: Here is a small sample o...,1
4,17790,30-10-2019 02:31,b'RT @MiaStJohnBoxer: After I lost my son &amp...,0


In [46]:
# Atribui labels aos tweets depressivos
labels_c = [x for x in df_tweets["label"]]

In [47]:
# Dados de treino
dados_treino = data_c[0:int(5000 * TRAIN_SPLIT)]
labels_treino = labels_c[0:int(5000 * TRAIN_SPLIT)]

In [48]:
# Os dados de treino são representados por uma matriz numérica (chamada embeddings) com valores que contém o 
# significado das palavras e do contexto.
dados_treino

array([[    0,     0,     0, ...,     4,     3,  9816],
       [    0,     0,     0, ...,     6,     7,     8],
       [    0,     0,     0, ...,     4,     3,  5293],
       ...,
       [    0,     0,     0, ...,     4,     3,   113],
       [    0,     0,     0, ...,     4,     3, 12793],
       [    0,     0,     0, ...,    20,    38,   112]], dtype=int32)

In [49]:
# Os labels estão definidos como 0 (não é depressivo) e 1 (é depressivo)
labels_treino

[1,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,


In [50]:
# Dados de teste
dados_teste = data_c[int(5000 * TRAIN_SPLIT):int(5000 * (TEST_SPLIT + TRAIN_SPLIT))]
labels_teste = labels_c[int(5000 * TRAIN_SPLIT):int(5000 * (TEST_SPLIT + TRAIN_SPLIT))]

In [51]:
# Dados de validação
dados_valid = data_c[int(5000 * (TEST_SPLIT + TRAIN_SPLIT)):5000]
labels_valid = labels_c[int(5000 * (TEST_SPLIT + TRAIN_SPLIT)):5000]

## Criação do Modelo de Classificação

Para o modelo, vamos usar uma arquitetura híbrida com CNN e LSTM. As camadas CNN vão aprender os detalhes dos dados e as camadas LSTM vão aprender a sequência, importante quando trabalhamos com texto.

In [52]:
# Função para o modelo
def modeloCNNLSTM(embedding_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH):
    
    # Cria a sequência de camadas
    model = Sequential()
    
    # A camada de entrada recebe as embeddings definidas anteriormente
    model.add(Embedding(len(embedding_matrix), 
                        EMBEDDING_DIM, 
                        weights = [embedding_matrix],
                        input_length = MAX_SEQUENCE_LENGTH, 
                        trainable = False))

    # Primeira camada convolucional com Max Pooling e Dropout para regularização
    model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 3))
    model.add(Dropout(0.3))

    # Segunda camada convolucional com Max Pooling e Dropout para regularização
    model.add(Conv1D(filters = 32, kernel_size = 3, padding = 'same', activation = 'relu'))
    model.add(MaxPooling1D(pool_size = 3))
    model.add(Dropout(0.3))

    # Camada LSTM
    model.add(LSTM(3500))
    model.add(Dropout(0.4))
    
    # Camada Densa (camada de saída)
    model.add(Dense(1, activation = 'sigmoid'))

    # Compila o modelo com função de custo, otimizador e métrica
    model.compile(loss = 'binary_crossentropy', optimizer = 'nadam', metrics = ['acc'])
    
    return model

In [53]:
# Cria o modelo
modelo = modeloCNNLSTM(embedding_matrix, EMBEDDING_DIM, MAX_SEQUENCE_LENGTH)

In [54]:
# Sumário do modelo
print(modelo.summary())

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 140, 300)          6000000   
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 140, 32)           28832     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 46, 32)            0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 46, 32)            0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 46, 32)            3104      
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, 15, 32)            0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 15, 32)           

In [55]:
# Vamos definir o EarlyStopping para que o treinamento seja finalizado se o erro não for reduzido em até 3 épocas
early_stop = EarlyStopping(monitor = 'val_loss', patience = 3)

In [56]:
# Compila novamente o modelo
modelo.compile(loss = 'binary_crossentropy', optimizer = 'nadam', metrics = ['acc'])

## Treinamento do Modelo

In [57]:
# Treina o modelo
print("\nIniciando o Treinamento. Aguarde!\n")

hist = modelo.fit(dados_treino, 
                  labels_treino, 
                  validation_data = (dados_valid, labels_valid), 
                  epochs = EPOCHS, 
                  batch_size = 100,
                  shuffle = True, 
                  callbacks = [early_stop])

print("\nTreinamento Concluído!")


Iniciando o Treinamento. Aguarde!

Train on 3500 samples, validate on 500 samples
Epoch 1/15
3500/3500 [==============================] - 5s 2ms/step - loss: 1.2568 - acc: 0.5991 - val_loss: 0.6192 - val_acc: 0.6480
Epoch 2/15
3500/3500 [==============================] - 3s 936us/step - loss: 0.4878 - acc: 0.7549 - val_loss: 0.3438 - val_acc: 0.8560
Epoch 3/15
3500/3500 [==============================] - 3s 938us/step - loss: 0.3230 - acc: 0.8674 - val_loss: 0.3237 - val_acc: 0.8640
Epoch 4/15
3500/3500 [==============================] - 3s 937us/step - loss: 0.2590 - acc: 0.9029 - val_loss: 0.2848 - val_acc: 0.8760
Epoch 5/15
3500/3500 [==============================] - 3s 938us/step - loss: 0.2246 - acc: 0.9140 - val_loss: 0.3334 - val_acc: 0.8680
Epoch 6/15
3500/3500 [==============================] - 3s 938us/step - loss: 0.2159 - acc: 0.9171 - val_loss: 0.2732 - val_acc: 0.8960
Epoch 7/15
3500/3500 [==============================] - 3s 939us/step - loss: 0.1819 - acc: 0.9326 - va

## Avaliação do Modelo

In [58]:
# Fazendo previsões com dados de teste
labels_c_pred = modelo.predict(dados_teste)

In [59]:
# Ajustando o shape
labels_pred = np.round(labels_c_pred.flatten())

In [60]:
# Calculando a acurácia
accuracy = accuracy_score(labels_teste, labels_pred)

In [61]:
print("Acurácia do Modelo: %.2f%%" % (accuracy * 100))

Acurácia do Modelo: 95.20%


In [62]:
# Relatório de Classificação
print(classification_report(labels_teste, labels_pred))

              precision    recall  f1-score   support

           0       0.96      0.92      0.94       418
           1       0.95      0.97      0.96       582

    accuracy                           0.95      1000
   macro avg       0.95      0.95      0.95      1000
weighted avg       0.95      0.95      0.95      1000



O modelo está bastante equilibrado e com boa performance. Nossa arquitetura foi bastante eficiente. Vamos salvar o modelo em disco.

In [63]:
# Nome do arquivo em disco
model_name = "modelo/modelo_acc={}".format(accuracy)

In [64]:
# Salva o modelo
modelo.save(model_name)

Feito. Temos um modelo capaz de detectar depressão em comentários em redes sociais!

A partir daqui você pode:
    
- 1- Alterar os hiperparâmetros e tentar melhorar a performance do modelo.
- 2- Aplicar o modelo a dados coletados do Twitter em tempo real (será necessário criar a API no site do Twitter). Nesse caso deverá ser definido algum critério para classificar os tweets como depressivos ou não.
- 3- Tentar aplicar o modelo a dados de outras redes sociais.

# Fim